In [1]:
weights = {'hp': 0.2, 'accel': 0.3, 'weight': 0.1, 'mpg': 0.4}

In [2]:
def js_formatter(js_master, value, weights):
    '''formats a JS code string with appropriate parameters'''
    values = list(weights.keys())
    values.remove(value)
    js_string = js_master % (value, weights[value], values[0], weights[values[0]], values[1], weights[values[1]], weights[values[2]], weights[values[2]], colors['red'], colors['yellow'], colors['green'])
    
    return js_string

In [3]:
jscode_master = """
        var data = source.data;
        var v = cb_obj.value

        var hp = data['hp']
        var accel = data['accel']
        var weight = data['weight']
        var mpg = data['mpg']
    
        var label = data['label']
        var color = data['color']
    
        %s[0] = v

        var score =  v * %s + %s * %s + %s * %s + %s * %s

        label[0] = score.toFixed(1)

        if (score < 4) {
            color[0]  = '%s'
        }
        else if (score < 8) {
        color[0]  = '%s'
        }
        else {
            color[0]  = '%s'
        }
    
        source.change.emit();
    """

In [5]:
colors = {'white': '#ffffff', 'gray': '#888888', 'red': '#E8898E', 'yellow': '#ECBA91', 'green': '#9BC4AF'}

In [7]:
js = js_formatter(jscode_master, 'hp', weights)

In [9]:
print(js)


        var data = source.data;
        var v = cb_obj.value

        var hp = data['hp']
        var accel = data['accel']
        var weight = data['weight']
        var mpg = data['mpg']
    
        var label = data['label']
        var color = data['color']
    
        hp[0] = v

        var score =  v * 0.2 + accel * 0.3 + weight * 0.1 + 0.4 * 0.4

        label[0] = score.toFixed(1)

        if (score < 4) {
            color[0]  = '#E8898E'
        }
        else if (score < 8) {
        color[0]  = '#ECBA91'
        }
        else {
            color[0]  = '#9BC4AF'
        }
    
        source.change.emit();
    


In [1]:
import sqlite3 as sqlite

In [2]:
import pandas as pd

In [3]:
conn = sqlite.connect('data/cars.db')

In [4]:
data = pd.read_sql_query('''SELECT * FROM cars''', conn)

In [5]:
data

,index,brand,model,model_year,origin,cylinders,displacement_ccm,horsepower,acceleration,weight_kg,liters_per_100km
0,0,CHEVROLET,CHEVELLE MALIBU,1970,AMERICA,8,5030.828648,130.0,12.0,1589.387664,13.055556
1,1,BUICK,SKYLARK 320,1970,AMERICA,8,5735.472400,165.0,11.5,1675.116622,15.666667
2,2,PLYMOUTH,SATELLITE,1970,AMERICA,8,5211.086352,150.0,11.0,1558.543383,13.055556
3,3,AMC,REBEL SST,1970,AMERICA,8,4981.667456,150.0,12.0,1557.182606,14.687500
4,4,FORD,TORINO,1970,AMERICA,8,4948.893328,140.0,10.5,1564.440084,13.823529
...,...,...,...,...,...,...,...,...,...,...,...
393,393,FORD,MUSTANG GL,1982,AMERICA,4,2294.188960,86.0,15.6,1265.522712,8.703704
394,394,VOLKSWAGEN,PICKUP,1982,EUROPE,4,1589.545208,52.0,24.6,966.151748,5.340909
395,395,DODGE,RAMPAGE,1982,AMERICA,4,2212.253640,84.0,11.6,1040.994489,7.343750
396,396,FORD,RANGER,1982,AMERICA,4,1966.447680,79.0,18.6,1190.679971,8.392857


In [24]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             398 non-null    int64  
 1   brand             398 non-null    object 
 2   model             398 non-null    object 
 3   model_year        398 non-null    int64  
 4   origin            398 non-null    object 
 5   cylinders         398 non-null    int64  
 6   displacement_ccm  398 non-null    float64
 7   horsepower        398 non-null    float64
 8   acceleration      398 non-null    float64
 9   weight_kg         398 non-null    float64
 10  liters_per_100km  398 non-null    float64
dtypes: float64(5), int64(3), object(3)
memory usage: 34.3+ KB


In [25]:
data.loc[(data['model_year'] == 1977) & (data['origin']  == 'AMERICA'), ['cylinders', 'horsepower', 'acceleration', 'weight_kg', 'liters_per_100km']].mean()

cylinders              6.222222
horsepower           118.388889
acceleration          15.238889
weight_kg           1552.193090
liters_per_100km      12.128730
dtype: float64

In [28]:
def assign_points(df, col, points, reverse = False):    
    points_col = '{}_points'.format(col) 
    
    if reverse == True:    
        for w in points.keys():    
            df.loc[df[col] > w, points_col] = points[w]
    else: 
        for w in points.keys():    
            df.loc[df[col] < w, points_col] = points[w]
            
    df.loc[df[points_col].isna(), points_col] = 1
    return df

In [20]:
def assign_cyl_points(df):
    for c, p in zip([6,8,4], [5,4,3]):
        df.loc[df['cylinders'] == c, 'cylinders_points'] = p
    return df

In [21]:
points = {'acceleration': {14.7:2, 13.8:3, 13:4, 11.3:5},
        'liters_per_100km': {9.4:2, 8.1:3, 7.3:4, 6.4:5},
        'weight_kg': {1170: 2, 1010: 3, 950: 4, 870: 5}, 
        'horsepower': {100: 2, 125: 3, 150: 4, 180: 5}}

In [30]:
data = assign_points(data, 'horsepower', points['horsepower'], reverse=True)


In [31]:
data = assign_points(data, 'acceleration', points['acceleration'])

In [32]:
data = assign_points(data, 'weight_kg', points['weight_kg'])

In [33]:
data = assign_points(data, 'liters_per_100km', points['liters_per_100km'])

In [34]:
data = assign_cyl_points(data)

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    398 non-null    int64  
 1   brand                    398 non-null    object 
 2   model                    398 non-null    object 
 3   model_year               398 non-null    int64  
 4   origin                   398 non-null    object 
 5   cylinders                398 non-null    int64  
 6   displacement_ccm         398 non-null    float64
 7   horsepower               398 non-null    float64
 8   acceleration             398 non-null    float64
 9   weight_kg                398 non-null    float64
 10  liters_per_100km         398 non-null    float64
 11  horsepower_points        398 non-null    float64
 12  acceleration_points      398 non-null    float64
 13  weight_kg_points         398 non-null    float64
 14  liters_per_100km_points  3

In [51]:
for origin in ['AMERICA', 'EUROPE', 'ASIA']:
    for year in range(1970, 1983):    
        
        query = data.loc[(data['model_year'] == year) & (data['origin']  == origin), ['liters_per_100km_points', 'acceleration_points', 'horsepower_points', 'weight_kg_points', ]].mean()
        value = sum([item*weight for item, weight in zip(query, [0.4, 0.3, 0.2, 0.1])])
        print(year, origin, [round(item, 1) for item in query], round(value, 1))

1970 AMERICA [1.0, 3.5, 3.7, 1.0] 2.3
1971 AMERICA [1.1, 2.3, 2.2, 1.4] 1.7
1972 AMERICA [1.1, 2.6, 2.9, 1.4] 1.9
1973 AMERICA [1.0, 2.9, 3.0, 1.1] 2.0
1974 AMERICA [1.1, 1.3, 1.8, 1.3] 1.3
1975 AMERICA [1.0, 1.6, 1.9, 1.0] 1.4
1976 AMERICA [1.1, 1.9, 2.0, 1.4] 1.6
1977 AMERICA [1.5, 1.9, 2.2, 1.5] 1.8
1978 AMERICA [1.4, 1.8, 1.9, 1.3] 1.6
1979 AMERICA [1.7, 1.8, 1.9, 1.4] 1.7
1980 AMERICA [1.8, 1.2, 1.2, 1.3] 1.5
1981 AMERICA [2.4, 1.5, 1.2, 1.9] 1.9
1982 AMERICA [2.6, 1.4, 1.1, 1.4] 1.8
1970 EUROPE [1.4, 1.8, 1.2, 2.4] 1.6
1971 EUROPE [2.5, 1.5, 1.0, 4.0] 2.0
1972 EUROPE [1.2, 1.2, 1.2, 1.8] 1.3
1973 EUROPE [1.4, 1.3, 1.3, 2.4] 1.5
1974 EUROPE [2.0, 1.3, 1.0, 3.0] 1.7
1975 EUROPE [1.2, 1.7, 1.2, 1.8] 1.4
1976 EUROPE [1.6, 1.5, 1.2, 2.5] 1.6
1977 EUROPE [2.5, 2.2, 1.2, 3.2] 2.2
1978 EUROPE [2.0, 1.3, 1.8, 2.0] 1.8
1979 EUROPE [3.0, 1.2, 1.0, 2.2] 2.0
1980 EUROPE [4.1, 1.0, 1.0, 2.6] 2.4
1981 EUROPE [3.2, 1.5, 1.0, 2.0] 2.1
1982 EUROPE [4.5, 1.0, 1.0, 3.5] 2.7
1970 ASIA [1.5, 1.5, 1.0,

In [6]:
data.describe()

,index,model_year,cylinders,displacement_ccm,horsepower,acceleration,weight_kg,liters_per_100km
count,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000
mean,198.500000,1976.010050,5.449749,3166.306036,104.140704,15.526633,1346.711188,11.189500
std,115.036951,3.697627,1.702343,1708.386502,38.287265,2.741884,384.303067,3.908365
min,0.000000,1970.000000,3.000000,1114.320352,46.000000,8.000000,731.644493,5.042918
25%,99.250000,1973.000000,4.000000,1720.641720,75.000000,13.800000,1007.315256,8.103448
50%,198.500000,1976.000000,4.000000,2392.511344,92.000000,15.500000,1266.656693,10.217391
75%,297.750000,1979.000000,8.000000,4293.410768,125.000000,17.000000,1636.561271,13.428571
max,397.000000,1982.000000,8.000000,7456.114120,230.000000,24.800000,2331.464782,26.111111
